## 滑动平均模型使模型在测试数据集上更健壮

- 衰减率（decay）：控制模型的更新速度
- 影子变量（shadow variable）：初始值就是相应变量的初始值，而每次运行变量更新时，影子变量的值会更新为：
![影子变量更新公式](pic/滑动平均模型_1.png)
variable为待更新变量，从公式中可以看出decay越大模型越稳定，一般取0.999。为了使模型在训练前期可以更新的更快，num_updates参数可以动态设置decay的大小。如果初始化时提供了num_updates，那么每次使用的衰减率为：
![衰减率更新公式](pic/滑动平均模型_2.png)

In [1]:
import tensorflow as tf

In [2]:
v1 = tf.Variable(0, dtype=tf.float32)
step = tf.Variable(0, trainable=False)                 # 模拟神经网络模型中迭代的轮数
ema = tf.train.ExponentialMovingAverage(0.99, step)    # 定义一个滑动平均的类
maintain_averages_op = ema.apply([v1])                 # 定义一个更新滑动平均的操作。这里需要给定一个列表，每次执行这个操作时
                                                       # 这个列表中的变量会被更新。

In [3]:
with tf.Session() as sess:
    
    # 初始化
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    print(sess.run([v1, ema.average(v1)]))
    
    # 更新变量v1的取值到5。
    sess.run(tf.assign(v1, 5))
    # 更新v1的滑动平均值。衰减率为min{0.99, (1+step)/(10+step)=0.1} = 0.1
    # 所以v1的滑动平均会被更新为0.1*0+0.9*5=4.5
    sess.run(maintain_averages_op)
    print(sess.run([v1, ema.average(v1)]))
    
    # 更新step和v1的取值
    sess.run(tf.assign(step, 10000))  
    sess.run(tf.assign(v1, 10))
    # 更新v1的滑动平均值。衰减率为min{0.99, (1+step)/(10+step)=0.999} = 0.99
    # 所以v1的滑动平均会被更新为0.99*4.5+0.01*10=4.555
    sess.run(maintain_averages_op)
    print(sess.run([v1, ema.average(v1)]))       
    
    # 再次更新v1的滑动平均值，得到新的滑动平均值0.99*4.555+0.01*10=4.60945
    sess.run(maintain_averages_op)
    print(sess.run([v1, ema.average(v1)]))

[0.0, 0.0]
[5.0, 4.5]
[10.0, 4.555]
[10.0, 4.60945]
